# GIFT-Riemann Graph Structure Analysis

## Discovery

The algebraic relations among Riemann zeros form a **dependency graph**:
- Each relation `a×γᵢ² - b×γⱼ² + γₖ + c ≈ 0` connects nodes i, j, k
- Both coefficients (a, b) AND indices (i, j, k) are GIFT constants

This notebook explores:
1. **Which indices are hubs?** — Most connected zeros
2. **Clustering** — Do relations form communities?
3. **Prediction graph** — Can we predict γₙ from earlier zeros?
4. **Recurrence structure** — Is there a GIFT-based recursion?

---

In [ ]:
# ============================================================
# SETUP
# ============================================================
!pip install networkx cupy-cuda12x -q

import numpy as np
import cupy as cp
import networkx as nx
from collections import Counter, defaultdict
from itertools import combinations, product
import json
import time

print("GPU:", cp.cuda.runtime.getDeviceProperties(0)['name'].decode())

In [ ]:
# ============================================================
# RIEMANN ZEROS
# ============================================================
GAMMA = np.array([
    14.134725141734693, 21.022039638771555, 25.010857580145688,
    30.424876125859513, 32.935061587739189, 37.586178158825671,
    40.918719012147495, 43.327073280914999, 48.005150881167159,
    49.773832477672302, 52.970321477714460, 56.446247697063394,
    59.347044002602353, 60.831778524609809, 65.112544048081607,
    67.079810529494173, 69.546401711173979, 72.067157674481907,
    75.704690699083933, 77.144840068874805, 79.337375020249367,
    82.910380854086030, 84.735492980517050, 87.425274613125229,
    88.809111207634465, 92.491899270558484, 94.651344040519848,
    95.870634228245309, 98.831194218193692, 101.31785100573139,
    103.72553804047833, 105.44662305232609, 107.16861118427640,
    111.02953554316967, 111.87465917699263, 114.32022091545271,
    116.22668032085755, 118.79078286597621, 121.37012500242064,
    122.94682929355258, 124.25681855434864, 127.51668387959649,
    129.57870419995605, 131.08768853093265, 133.49773720299758,
    134.75650975337387, 138.11604205453344, 139.73620895212138,
    141.12370740402112, 143.11184580762063
])

g = lambda n: GAMMA[n-1]
print(f"Loaded {len(GAMMA)} zeros")

In [ ]:
# ============================================================
# GIFT CONSTANTS
# ============================================================

GIFT_COEFFS = {
    'N_gen': 3, 'h_G2': 6, 'dim_K7': 7, 'rank_E8': 8,
    'h_F4': 12, 'dim_G2': 14, 'h_E7': 18, 'b2': 21,
    'H_star-b3': 22, 'dim_J3O': 27, 'h_E8': 30, 'L8': 47,
    'dim_K7_sq': 49, 'b3-b2': 56, 'b3-dim_G2': 63, 'b3': 77, 'H_star': 99
}

# Indices that are themselves GIFT constants
GIFT_INDICES = {
    3: 'N_gen', 6: 'h_G2', 7: 'dim_K7', 8: 'rank_E8',
    12: 'h_F4', 14: 'dim_G2', 18: 'h_E7', 21: 'b2',
    22: 'H_star-b3', 27: 'dim_J3O', 30: 'h_E8', 47: 'L8'
}

print(f"GIFT coefficients: {len(GIFT_COEFFS)}")
print(f"GIFT indices (in range 1-50): {list(GIFT_INDICES.keys())}")

In [ ]:
# ============================================================
# BUILD RELATION DATABASE (GPU-accelerated)
# ============================================================

def find_all_relations_gpu(gamma, gift_coeffs, max_idx=50, threshold=0.001):
    """
    Find all relations: a×γᵢ² - b×γⱼ² + γₖ + c ≈ 0
    Returns list of (i, j, k, a_name, b_name, c, error)
    """
    gamma_gpu = cp.array(gamma[:max_idx])
    gamma_sq = gamma_gpu ** 2
    
    relations = []
    coeff_items = [(n, v) for n, v in gift_coeffs.items() if v <= 100]
    
    # Precompute all a×γᵢ² and b×γⱼ²
    for a_name, a in coeff_items:
        for b_name, b in coeff_items:
            # Compute a×γᵢ² - b×γⱼ² for all i,j pairs
            a_gamma_sq = a * gamma_sq  # shape (max_idx,)
            b_gamma_sq = b * gamma_sq
            
            # diff[i,j] = a×γᵢ² - b×γⱼ²
            diff = a_gamma_sq[:, None] - b_gamma_sq[None, :]  # (max_idx, max_idx)
            
            for c in [-1, 0, 1]:
                # target[i,j,k] = a×γᵢ² - b×γⱼ² + γₖ + c
                # We want this ≈ 0
                for k in range(max_idx):
                    target = diff + gamma_gpu[k] + c  # (max_idx, max_idx)
                    rel_err = cp.abs(target) / (a * gamma_sq[:, None])
                    
                    # Find pairs where error < threshold
                    good = cp.where(rel_err < threshold)
                    
                    for idx in range(len(good[0])):
                        i = int(good[0][idx]) + 1  # 1-indexed
                        j = int(good[1][idx]) + 1
                        if i != j and i != k+1 and j != k+1:
                            err = float(rel_err[good[0][idx], good[1][idx]]) * 100
                            relations.append({
                                'i': i, 'j': j, 'k': k+1,
                                'a': a, 'a_name': a_name,
                                'b': b, 'b_name': b_name,
                                'c': c,
                                'error': err
                            })
    
    return relations

print("Building relation database (GPU)...")
t0 = time.time()
all_relations = find_all_relations_gpu(GAMMA, GIFT_COEFFS, max_idx=50, threshold=0.0005)
print(f"Found {len(all_relations)} relations in {time.time()-t0:.1f}s")

In [ ]:
# ============================================================
# BUILD THE DEPENDENCY GRAPH
# ============================================================

G = nx.Graph()

# Add all indices as nodes
for n in range(1, 51):
    is_gift = n in GIFT_INDICES
    G.add_node(n, gift_index=is_gift, gift_name=GIFT_INDICES.get(n, ''))

# Add edges from relations
edge_weights = defaultdict(float)
for rel in all_relations:
    i, j, k = rel['i'], rel['j'], rel['k']
    # Weight inversely proportional to error (better relations = stronger edges)
    weight = 1.0 / (rel['error'] + 0.0001)
    
    # Add edges between all pairs in the relation
    edge_weights[(min(i,j), max(i,j))] += weight
    edge_weights[(min(i,k), max(i,k))] += weight
    edge_weights[(min(j,k), max(j,k))] += weight

for (u, v), w in edge_weights.items():
    G.add_edge(u, v, weight=w)

print(f"Graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

In [ ]:
# ============================================================
# ANALYZE HUB STRUCTURE
# ============================================================

print("="*60)
print("HUB ANALYSIS: Most Connected Indices")
print("="*60)

# Degree centrality
degree_cent = nx.degree_centrality(G)
sorted_by_degree = sorted(degree_cent.items(), key=lambda x: -x[1])

print(f"\n{'Index':<8} {'Degree':<10} {'Centrality':<12} {'GIFT?':<15}")
print("-"*50)
for idx, cent in sorted_by_degree[:20]:
    deg = G.degree(idx)
    gift = GIFT_INDICES.get(idx, '')
    print(f"{idx:<8} {deg:<10} {cent:<12.4f} {gift:<15}")

# Count GIFT indices in top 10
top10_indices = [x[0] for x in sorted_by_degree[:10]]
gift_in_top10 = sum(1 for i in top10_indices if i in GIFT_INDICES)
print(f"\nGIFT indices in top 10: {gift_in_top10}/10")

In [ ]:
# ============================================================
# WEIGHTED CENTRALITY (by relation precision)
# ============================================================

print("\n" + "="*60)
print("WEIGHTED CENTRALITY (precision-weighted)")
print("="*60)

# Eigenvector centrality (considers edge weights)
eigen_cent = nx.eigenvector_centrality(G, weight='weight', max_iter=1000)
sorted_by_eigen = sorted(eigen_cent.items(), key=lambda x: -x[1])

print(f"\n{'Index':<8} {'Eigen Cent':<12} {'GIFT?':<15} {'γₙ value':<12}")
print("-"*50)
for idx, cent in sorted_by_eigen[:15]:
    gift = GIFT_INDICES.get(idx, '')
    print(f"{idx:<8} {cent:<12.6f} {gift:<15} {GAMMA[idx-1]:<12.2f}")

In [ ]:
# ============================================================
# COMMUNITY DETECTION
# ============================================================

print("\n" + "="*60)
print("COMMUNITY STRUCTURE")
print("="*60)

# Louvain communities
communities = nx.community.louvain_communities(G, weight='weight', seed=42)

print(f"\nFound {len(communities)} communities")
for i, comm in enumerate(sorted(communities, key=len, reverse=True)):
    comm_list = sorted(comm)
    gift_members = [n for n in comm_list if n in GIFT_INDICES]
    print(f"\nCommunity {i+1} ({len(comm)} members):")
    print(f"  Indices: {comm_list[:15]}{'...' if len(comm_list) > 15 else ''}")
    print(f"  GIFT members: {gift_members} ({[GIFT_INDICES[n] for n in gift_members]})")

In [ ]:
# ============================================================
# PREDICTION GRAPH: Can γₙ be predicted from earlier zeros?
# ============================================================

print("\n" + "="*60)
print("PREDICTION STRUCTURE")
print("="*60)

# For each n, find relations where n = max(i,j,k)
# These allow predicting γₙ from earlier zeros

predictable = {}
for rel in all_relations:
    i, j, k = rel['i'], rel['j'], rel['k']
    max_idx = max(i, j, k)
    
    if max_idx not in predictable:
        predictable[max_idx] = []
    predictable[max_idx].append(rel)

print(f"\n{'n':<5} {'# Relations':<12} {'Best Error %':<15} {'Best Predictors':<30}")
print("-"*65)

for n in range(10, 51):
    if n in predictable:
        rels = predictable[n]
        best = min(rels, key=lambda x: x['error'])
        predictors = sorted([best['i'], best['j'], best['k']])
        predictors = [p for p in predictors if p != n]
        print(f"{n:<5} {len(rels):<12} {best['error']:<15.6f} {predictors}")

In [ ]:
# ============================================================
# SEARCH FOR RECURRENCE PATTERNS
# ============================================================

print("\n" + "="*60)
print("RECURRENCE PATTERNS")
print("="*60)

# Look for relations of form: γₙ depends on γₙ₋ₖ for fixed k
lag_counts = Counter()

for rel in all_relations:
    indices = sorted([rel['i'], rel['j'], rel['k']])
    max_idx = indices[2]
    
    # Lags from max to others
    for idx in indices[:2]:
        lag = max_idx - idx
        lag_counts[lag] += 1

print(f"\nMost common lags (n - predictors):")
print(f"{'Lag':<8} {'Count':<10} {'GIFT meaning'}")
print("-"*40)
for lag, count in lag_counts.most_common(20):
    gift_meaning = ''
    if lag == 3:
        gift_meaning = 'N_gen'
    elif lag == 6:
        gift_meaning = 'h_G2'
    elif lag == 7:
        gift_meaning = 'dim_K7'
    elif lag == 8:
        gift_meaning = 'rank_E8'
    elif lag == 14:
        gift_meaning = 'dim_G2'
    elif lag == 21:
        gift_meaning = 'b2'
    print(f"{lag:<8} {count:<10} {gift_meaning}")

In [ ]:
# ============================================================
# THE GIFT-LAG RECURRENCE HYPOTHESIS
# ============================================================

print("\n" + "="*60)
print("TESTING: γₙ = f(γₙ₋₃, γₙ₋ₖ) where k is GIFT")
print("="*60)

# The golden relation has step 3 = N_gen
# Test if there's a systematic recurrence with lag 3

def find_recurrence_with_lag(gamma, lag, max_n=50):
    """
    Find best relation: a×γₙ² - b×γₙ₋ₗₐ² + γₘ + c ≈ 0
    """
    results = []
    
    for n in range(lag + 1, max_n + 1):
        gn = gamma[n-1]
        gn_lag = gamma[n-lag-1]
        
        for a in range(1, 10):
            for b in range(1, 10):
                # Solve for γₘ: γₘ = b×γₙ₋ₗₐ² - a×γₙ² - c
                for c in [-1, 0, 1]:
                    target = b * gn_lag**2 - a * gn**2 - c
                    
                    # Find closest γₘ
                    diffs = np.abs(gamma[:max_n] - target)
                    m = np.argmin(diffs) + 1
                    error = diffs[m-1] / abs(target) * 100 if target != 0 else 999
                    
                    if error < 0.1 and m != n and m != n - lag:
                        results.append({
                            'n': n, 'lag': lag, 'm': m,
                            'a': a, 'b': b, 'c': c,
                            'error': error
                        })
    
    return results

# Test lag = 3 (N_gen)
recurrence_3 = find_recurrence_with_lag(GAMMA, lag=3, max_n=50)
print(f"\nLag 3 (N_gen): Found {len(recurrence_3)} recurrences")
if recurrence_3:
    for r in sorted(recurrence_3, key=lambda x: x['error'])[:10]:
        print(f"  n={r['n']}: {r['a']}γₙ² - {r['b']}γₙ₋₃² + γ_{r['m']} + {r['c']} ≈ 0 (err={r['error']:.4f}%)")

# Test lag = 6 (h_G2)
recurrence_6 = find_recurrence_with_lag(GAMMA, lag=6, max_n=50)
print(f"\nLag 6 (h_G2): Found {len(recurrence_6)} recurrences")
if recurrence_6:
    for r in sorted(recurrence_6, key=lambda x: x['error'])[:10]:
        print(f"  n={r['n']}: {r['a']}γₙ² - {r['b']}γₙ₋₆² + γ_{r['m']} + {r['c']} ≈ 0 (err={r['error']:.4f}%)")

In [ ]:
# ============================================================
# SPECIAL INDICES ANALYSIS
# ============================================================

print("\n" + "="*60)
print("SPECIAL INDICES: How often do GIFT indices appear?")
print("="*60)

index_freq = Counter()
for rel in all_relations:
    index_freq[rel['i']] += 1
    index_freq[rel['j']] += 1
    index_freq[rel['k']] += 1

print(f"\n{'Index':<8} {'Frequency':<12} {'GIFT?':<15} {'γₙ':<12} {'round(γₙ)':<10}")
print("-"*60)
for idx, freq in index_freq.most_common(25):
    gift = GIFT_INDICES.get(idx, '')
    gamma_val = GAMMA[idx-1]
    print(f"{idx:<8} {freq:<12} {gift:<15} {gamma_val:<12.2f} {round(gamma_val):<10}")

# Statistical test: are GIFT indices overrepresented?
total_appearances = sum(index_freq.values())
gift_appearances = sum(index_freq[i] for i in GIFT_INDICES if i in index_freq)
expected_gift = total_appearances * len(GIFT_INDICES) / 50

print(f"\nGIFT index appearances: {gift_appearances}")
print(f"Expected (random): {expected_gift:.0f}")
print(f"Enrichment factor: {gift_appearances / expected_gift:.2f}x")

In [ ]:
# ============================================================
# VISUALIZATION
# ============================================================
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# 1. Degree distribution
ax1 = axes[0, 0]
degrees = [G.degree(n) for n in G.nodes()]
gift_nodes = list(GIFT_INDICES.keys())
non_gift_nodes = [n for n in G.nodes() if n not in GIFT_INDICES]

ax1.bar([n for n in G.nodes()], degrees, color=['red' if n in GIFT_INDICES else 'steelblue' for n in G.nodes()], alpha=0.7)
ax1.set_xlabel('Index n')
ax1.set_ylabel('Degree (# connections)')
ax1.set_title('Node Degree Distribution (red = GIFT indices)')
ax1.grid(True, alpha=0.3)

# 2. Index frequency in relations
ax2 = axes[0, 1]
indices = list(range(1, 51))
freqs = [index_freq.get(i, 0) for i in indices]
colors = ['red' if i in GIFT_INDICES else 'steelblue' for i in indices]
ax2.bar(indices, freqs, color=colors, alpha=0.7)
ax2.set_xlabel('Index n')
ax2.set_ylabel('Frequency in relations')
ax2.set_title('Index Frequency in Relations (red = GIFT)')
ax2.grid(True, alpha=0.3)

# 3. Lag distribution
ax3 = axes[1, 0]
lags = list(range(1, 30))
lag_freqs = [lag_counts.get(l, 0) for l in lags]
gift_lags = [3, 6, 7, 8, 14, 21]
colors = ['red' if l in gift_lags else 'steelblue' for l in lags]
ax3.bar(lags, lag_freqs, color=colors, alpha=0.7)
ax3.set_xlabel('Lag (n - predictor index)')
ax3.set_ylabel('Frequency')
ax3.set_title('Lag Distribution (red = GIFT lags: 3,6,7,8,14,21)')
ax3.grid(True, alpha=0.3)

# 4. Network visualization (simplified)
ax4 = axes[1, 1]
# Use spring layout
pos = nx.spring_layout(G, k=2, iterations=50, seed=42)

# Draw non-GIFT nodes
nx.draw_networkx_nodes(G, pos, nodelist=non_gift_nodes, node_color='lightblue', 
                       node_size=100, ax=ax4, alpha=0.6)
# Draw GIFT nodes
nx.draw_networkx_nodes(G, pos, nodelist=gift_nodes, node_color='red', 
                       node_size=200, ax=ax4, alpha=0.8)
# Draw edges (only strong ones)
strong_edges = [(u, v) for u, v, d in G.edges(data=True) if d['weight'] > np.percentile([d['weight'] for _, _, d in G.edges(data=True)], 90)]
nx.draw_networkx_edges(G, pos, edgelist=strong_edges, alpha=0.3, ax=ax4)
# Labels for GIFT nodes
labels = {n: str(n) for n in gift_nodes}
nx.draw_networkx_labels(G, pos, labels, font_size=8, ax=ax4)

ax4.set_title('Relation Network (red = GIFT indices, top 10% edges)')
ax4.axis('off')

plt.tight_layout()
plt.savefig('gift_riemann_graph_structure.png', dpi=150)
plt.show()

print("\n✓ Figure saved to gift_riemann_graph_structure.png")

In [ ]:
# ============================================================
# SUMMARY
# ============================================================

summary = {
    'total_relations': len(all_relations),
    'graph_nodes': G.number_of_nodes(),
    'graph_edges': G.number_of_edges(),
    'top_hubs': [{'index': idx, 'degree': G.degree(idx), 'gift': idx in GIFT_INDICES} 
                for idx, _ in sorted_by_degree[:10]],
    'communities': len(communities),
    'gift_enrichment': float(gift_appearances / expected_gift),
    'top_lags': lag_counts.most_common(10),
    'recurrence_lag3_count': len(recurrence_3),
    'recurrence_lag6_count': len(recurrence_6)
}

print("\n" + "="*60)
print(" GRAPH STRUCTURE SUMMARY ")
print("="*60)
print(f"\nTotal relations found: {summary['total_relations']}")
print(f"Graph: {summary['graph_nodes']} nodes, {summary['graph_edges']} edges")
print(f"Communities detected: {summary['communities']}")
print(f"GIFT index enrichment: {summary['gift_enrichment']:.2f}x expected")
print(f"\nTop hubs (most connected):")
for h in summary['top_hubs'][:5]:
    gift_str = f"({GIFT_INDICES[h['index']]})" if h['gift'] else ""
    print(f"  Index {h['index']} {gift_str}: degree {h['degree']}")

print(f"\nMost common lags:")
for lag, count in summary['top_lags'][:5]:
    gift_str = '(GIFT)' if lag in [3, 6, 7, 8, 14, 21] else ''
    print(f"  Lag {lag} {gift_str}: {count} occurrences")

with open('gift_riemann_graph_summary.json', 'w') as f:
    json.dump(summary, f, indent=2, default=int)

print("\n✓ Summary saved to gift_riemann_graph_summary.json")